In [1]:
import sys
sys.path.append('..')
from Comms_System import Comms_System
import numpy as np
import matplotlib.pyplot as plt
import torch
from objective_functions import crossEntropy
from NetworkPytorch import train_loop, train_loop_old
from DE_Pytorch import DE
from operator import itemgetter
import copy

In [2]:
def get_data(num_symbols, sigma):
    
    symbol_set = [3, 1, -1, -3,] # all symbols that we use
    symbol_seq = np.random.choice(symbol_set, num_symbols, replace=True)
    m = 8
    CS = Comms_System(symbol_set=symbol_set, symbol_seq=symbol_seq, num_samples=m)

    gain_factor = np.max(np.convolve(CS.h, CS.h))
    upsampled = CS.upsample(v=False)
    Tx = np.convolve(upsampled, CS.h)
    Tx = Tx + np.random.normal(0.0, sigma, Tx.shape)  # add gaussian noise
    
    X = torch.tensor(Tx)
    X = X.view(1, 1, -1).float() # reshape and cast to float so PyTorch understands it
    y = symbol_seq
    classes = np.array(symbol_set)
    num_classes = len(classes)

    class_idx = {v: i for i, v in enumerate(classes)}
    y_idx = np.array([class_idx[v] for v in y])
    y = torch.Tensor(y_idx)
    
    return X, y

def make_net():
    net = torch.nn.Sequential(torch.nn.Conv1d(1, 1, 64), torch.nn.Conv1d(1, 4, 8, stride=8))
    for param in net.parameters():
        param.requires_grad = False
    return net

In [3]:
# Create Data
Xtrain, ytrain = get_data(num_symbols=1000, sigma=2)
Xtest, ytest = get_data(num_symbols=1000, sigma=2)

# Create 1D Convolutional Neural Network with PyTorch and define optimizer and loss
NN = torch.nn.Sequential(torch.nn.Conv1d(1, 1, 64), torch.nn.Conv1d(1, 4, 8, stride=8))
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(NN.parameters(), lr=1e-3)
D = DE(objective_function=torch.nn.CrossEntropyLoss(), population_function=make_net, 
       X=Xtrain, y=ytrain, Xtest=Xtest, ytest=ytest, F=0.55, cr=0.85)

In [ ]:
# Backprop Training

testcosts, traincosts = train_loop(model=NN, optimizer=optimizer, cost=criterion, Xtrain=Xtrain, ytrain=ytrain, 
                                   Xtest=Xtest, ytest=ytest, epochs=100000, eval=True, plot_iteration=1000)

In [4]:
# DE Training

best_agent = D.evolution(num_epochs=2000, verbose=True, print_epoch=100)

0: cost= 1.41082
0: testcost= 1.39281

100: cost= 0.94758
100: testcost= 0.93336

200: cost= 0.72537
200: testcost= 0.70484

300: cost= 0.58292
300: testcost= 0.62238

400: cost= 0.49493
400: testcost= 0.50827

500: cost= 0.44618
500: testcost= 0.47124

600: cost= 0.44035
600: testcost= 0.48160

700: cost= 0.41636
700: testcost= 0.43454

800: cost= 0.41056
800: testcost= 0.46275

900: cost= 0.39220
900: testcost= 0.42705

1000: cost= 0.38983
1000: testcost= 0.42182

1100: cost= 0.37463
1100: testcost= 0.40081

1200: cost= 0.36479
1200: testcost= 0.40316

1300: cost= 0.35436
1300: testcost= 0.39249

1400: cost= 0.34967
1400: testcost= 0.38514

1500: cost= 0.33847
1500: testcost= 0.37904

1600: cost= 0.33847
1600: testcost= 0.37904

1700: cost= 0.32979
1700: testcost= 0.39496

1800: cost= 0.32614
1800: testcost= 0.37957

1900: cost= 0.32614
1900: testcost= 0.37957



In [ ]:
# Accuracy

torch.sum(D.best_agent.forward(Xtest).argmax(axis=1) == ytest)/len(ytest)

In [ ]:
#%%timeit -o -r 5
#best_agent = D.evolution(num_epochs=1)